In [1]:
import numpy as np
import nibabel as nib
import scipy.ndimage as ndi
from skimage import filters, measure, morphology
import matplotlib.pyplot as plt

Loading the ct images

In [2]:
def load_ct_image(filepath):
    ct_img = nib.load(filepath)
    ct_data = ct_img.get_fdata()
    return ct_data, ct_img.affine, ct_img.header

Segmenting the bones

In [3]:
def segment_bones(ct_data, femur_threshold=300, tibia_threshold=300):

    
    
    depth, height, width = ct_data.shape
    print(f"CT volume dimensions: {depth}x{height}x{width}")  #to get the image dimension 
    
    segmented_femur = np.zeros_like(ct_data, dtype=bool)
    segmented_tibia = np.zeros_like(ct_data, dtype=bool)
    

    print(f"Using threshold value: {femur_threshold} HU for bones")  #creating the initial bone mask using threshold
    bone_mask = ct_data > femur_threshold
    
    # Cleaning up the mask
    bone_mask = morphology.remove_small_objects(bone_mask, min_size=50)
    bone_mask = morphology.binary_closing(bone_mask, morphology.ball(1))
    
    # Finding a good slice to analyze (where both bones are visible)
    # Calculate sum across each slice to find slices with significant bone content
    slice_bone_content = np.sum(bone_mask, axis=(1, 2))
    significant_slices = np.where(slice_bone_content > 100)[0]
    
    if len(significant_slices) == 0:
        print("No significant bone content found. Adjusting threshold...")
        # If no significant bone content is found then go with the lower threshold
        bone_mask = ct_data > (femur_threshold - 100)
        bone_mask = morphology.remove_small_objects(bone_mask, min_size=50)
        slice_bone_content = np.sum(bone_mask, axis=(1, 2))
        significant_slices = np.where(slice_bone_content > 100)[0]
    
    print(f"Found {len(significant_slices)} slices with significant bone content")
    
    # If we still can't find significant slices, use anatomical approach
    if len(significant_slices) == 0:
        print("Still no significant bone content found. Using anatomical approach...")
        # Divide the volume into anatomical regions
        femur_region = slice(0, depth // 2)  # Upper half for femur
        tibia_region = slice(depth // 2, depth)  # Lower half for tibia
        
        # Apply different thresholds for each region
        segmented_femur[femur_region] = ct_data[femur_region] > femur_threshold
        segmented_tibia[tibia_region] = ct_data[tibia_region] > tibia_threshold
    else:
        # Analyze each significant slice to identify femur and tibia
        for slice_idx in significant_slices:
            slice_mask = bone_mask[slice_idx]
            labeled_slice, num_labels = measure.label(slice_mask, return_num=True)
            
            if num_labels < 1:
                continue
                
            # Get region properties
            regions = measure.regionprops(labeled_slice)
            
            # For simplicity with this particular CT orientation:
            # Based on the image shown, the femur appears on the right side (higher x values)
            # and tibia appears on the left side (lower x values)
            # This is specific to the current dataset orientation
            
            # Sort regions by centroid x-coordinate (horizontal position)
            regions.sort(key=lambda r: r.centroid[1], reverse=True)  # Sort right to left
            
            # Apply to specific regions (may need adjustment based on actual image orientation)
            for i, region in enumerate(regions):
                if i == 0:  # Right-most region (assumed to be femur in this orientation)
                    segmented_femur[slice_idx][labeled_slice == region.label] = True
                elif i == 1:  # Second right-most region (assumed to be tibia in this orientation)
                    segmented_tibia[slice_idx][labeled_slice == region.label] = True
    
    # Clean up the segmentation masks
    segmented_femur = morphology.binary_closing(segmented_femur, morphology.ball(2))
    segmented_tibia = morphology.binary_closing(segmented_tibia, morphology.ball(2))
    
    # Remove small disconnected components
    segmented_femur = morphology.remove_small_objects(segmented_femur, min_size=100)
    segmented_tibia = morphology.remove_small_objects(segmented_tibia, min_size=100)
    
    # If the femur or tibia mask is empty, try a different approach
    if not np.any(segmented_femur) or not np.any(segmented_tibia):
        print("Warning: One or both segmentation masks are empty. Trying alternative approach...")
        # Use anatomical knowledge about the rough positions of femur and tibia in this CT
        # For this specific orientation, divide the image into regions
        
        # For this specific dataset, create spatial masks based on the approximate locations
        # These values may need adjustment based on the actual data
        height_mid = height // 2
        width_mid = width // 2
        
        # Create spatial masks for femur (right upper quadrant) and tibia (right lower quadrant)
        femur_spatial_mask = np.zeros((depth, height, width), dtype=bool)
        tibia_spatial_mask = np.zeros((depth, height, width), dtype=bool)
        
        # Define spatial regions (adjust based on CT orientation)
        femur_spatial_mask[:, :height_mid, width_mid:] = True  # Upper right quadrant
        tibia_spatial_mask[:, height_mid:, width_mid:] = True  # Lower right quadrant
        
        # Combine spatial masks with intensity threshold
        segmented_femur = (ct_data > femur_threshold) & femur_spatial_mask
        segmented_tibia = (ct_data > tibia_threshold) & tibia_spatial_mask
        
        # Clean up
        segmented_femur = morphology.remove_small_objects(segmented_femur, min_size=100)
        segmented_tibia = morphology.remove_small_objects(segmented_tibia, min_size=100)
    
    print("Bone segmentation completed")
    return segmented_femur, segmented_tibia